In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re

params

In [ ]:
file_path = 'results/evaluation.json'  # Path to the results file
model_name = 'gpt-3.5'                   # Model name, e.g., 'gpt-4'
scoring_template = 'scoring_1_10_cot'      # Scoring template

In [ ]:
score_mapping = {
    "scoring_badges" : {
        'Novice': 10,
        'Apprentice': 9,
        'Journeyman': 8,
        'Craftsman': 7,
        'Artisan': 6,
        'Expert': 5,
        'Master': 4,
        'Sage': 3,
        'Oracle': 2,
        'Legend': 1
    },
    "scoring_grades" : {
        'Beginner': 5,
        'Intermediate': 4,
        'Proficient': 3, 
        'Advanced': 2,
        'Expert': 1  
    }
}

depending on the complexity of the llm eval score response you may need to extract the scores
manually and assign them to the `scores` list. below is an attempt to automate that process. 

read data (numeric labels)

In [ ]:
with open(file_path, 'r') as file:
    data_json = json.load(file)

eval_data = data_json[model_name][scoring_template]

misspelled_percentages = []
scores = []

for item in eval_data:
    misspelled_percentages.append(item["misspelled_percentage"])
    resp = item["response"]
    try:
        score = int(resp.split(":")[1].strip())
    except ValueError:
        numbers = re.findall(r'\d+', resp)
        score = int(numbers[-1])
    scores.append(score)

read data (classification labels)

In [ ]:
with open(file_path, 'r') as file:
    data_json = json.load(file)

eval_data = data_json[model_name][scoring_template]

misspelled_percentages = []
scores = []

for item in eval_data:
    misspelled_percentages.append(item["misspelled_percentage"])
    resp = item["response"]
    grade = resp.split()[-1].strip()
    score = score_mapping[scoring_template][grade]
    
    scores.append(score)

manual

In [ ]:
scores = []
misspelled_percentages = []

visualize

In [ ]:
sns.set(style="whitegrid", palette="muted")

fig = plt.figure(figsize=(12, 7))

sns.set(rc={'axes.facecolor':'#edf6fc'})
ax = sns.scatterplot(x=misspelled_percentages, y=scores, s=50, color="black", alpha=0.7)

plt.xlabel('Percentage of Misspelled Words (%)', fontsize=12, style='italic')
plt.ylabel('LLM Score', fontsize=12, style='italic')

plt.xticks([0,10, 20, 30, 40, 50, 60, 70, 80, 90, 100], fontsize=12)
plt.yticks([0,1,2,3,4,5,6,7,8,9,10], fontsize=12)
#plt.xticks([0, 25, 50, 75, 100], fontsize=12)
#plt.yticks([1, 2, 3, 4, 5], fontsize=12)

plt.ylim(bottom=-0.5)

ax.tick_params(axis='both', which='major', pad=10)

plt.grid(True, linestyle='--', linewidth=2, alpha=1)

#plt.show()
plt.savefig(f"figures/{model_name}_{scoring_template}.png")

r-squared and pearson correlation coefficient

In [ ]:
scores = [1, 4, 5, 5, 5]
misspelled_percentages = [1, 2, 3, 4, 5]

In [ ]:
from scipy.stats import pearsonr
import numpy as np

pearson_corr, _ = pearsonr(misspelled_percentages, scores)

slope = 0.1
line = slope * np.array(misspelled_percentages)
r_squared = np.corrcoef(scores, line)[0, 1]**2

pearson_corr, r_squared

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
categories = ['Correctly identified', 'Incorrectly identified']
counts = [10, 6]

# Creating the bar chart
plt.figure(figsize=(8, 6))
plt.yticks([0,1,2,3,4,5,6,7,8,9,10], fontsize=12)
plt.ylabel('# comparisons')
plt.bar(categories, counts, color=['green', 'red'], alpha=0)
plt.show()


In [ ]:

sns.set(style="whitegrid", palette="muted")

fig = plt.figure(figsize=(12, 7))

sns.set(rc={'axes.facecolor':'#edf6fc'})
ax = sns.barplot( color="black", alpha=0.7)

plt.bar(categories, counts, color=['green', 'red'], alpha=0)
plt.ylim(bottom=-0.5)

ax.xaxis.grid(False)  # Disable the x-axis grid lines
ax.yaxis.grid(True)  # Ensure the y-axis grid lines are visible


ax.tick_params(axis='both', which='major', pad=10)

plt.show()
#plt.savefig(f"figures/{model_name}_{scoring_template}.png")